In [1]:
import pandas as pd
import math
from dateutil.relativedelta import relativedelta
from datetime import datetime
import re
import copy

In [2]:
slab_fields = pd.read_excel(r'C:\Users\CDMA\Downloads\slab_fields_prep_v2.xlsx',sheet_name = 'slab_fields_prep')

In [3]:
slab_new = pd.read_csv(r'C:\Users\CDMA\Downloads\vm_tax_slab_new_transformed.csv')
slab_new = slab_new[slab_new['state_cd']=='AS']

In [1]:
slab_new['date_from'] = slab_new['date_from'].astype('datetime64[ns]')
slab_new['date_to'] = slab_new['date_to'].astype('datetime64[ns]')

NameError: name 'slab_new' is not defined

In [5]:
slab_add_new = pd.read_csv(r'D:\Jharkhand python\vm_tax_slab_add_new.csv')

In [6]:
va_details = pd.read_csv(r'D:\AS\va_details.csv')

FileNotFoundError: [Errno 2] File D:\AS\va_details.csv does not exist: 'D:\\AS\\va_details.csv'

In [21]:
vt_tax_exem = pd.read_csv(r'D:\AS\vt_tax_exem.csv')
vt_tax_exem['Perm Dt'] = vt_tax_exem.loc[:,'Perm Dt'].apply(lambda x: datetime.strptime(x[:11], '%d-%b-%Y'))

In [12]:
vt_tax_based_on = pd.read_csv(r'D:\AS\vt_tax_based_on.csv')
#vt_tax_based_on_permit_info = pd.read_csv(r'D:\Jharkhand python\vt_tax_based_on_permit_info.csv')

In [11]:
vt_tax = pd.read_csv(r'D:\AS\vt_tax.csv',nrows=1000)
vt_tax = vt_tax[vt_tax['Pur Cd']==58]
vt_tax = vt_tax[vt_tax['Tax Mode']!='B']
vt_tax=vt_tax[vt_tax['Collected By']!='VAHAN2']
vt_tax.reset_index(drop=True, inplace=True)

In [13]:
vt_owner = pd.read_csv(r'D:\AS\vt_owner.csv')

C:\Users\CDMA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
vp_appl_rcpt_mapping = pd.read_csv(r'D:\Jharkhand python\vp_appl_rcpt_mapping.csv')

C:\Users\CDMA\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [14]:
tm_office = pd.read_csv(r'D:\AS\tm_office.csv')
tm_office['State Cd'].unique

<bound method Series.unique of 0     AS
1     AS
2     AS
3     AS
4     AS
5     AS
6     AS
7     AS
8     AS
9     AS
10    AS
11    AS
12    AS
13    AS
14    AS
15    AS
16    AS
17    AS
18    AS
19    AS
20    AS
21    AS
22    AS
23    AS
24    AS
25    AS
26    AS
27    AS
28    AS
29    AS
Name: State Cd, dtype: object>

In [15]:
def condition_form_convert(cf_params,params_info, c_formula):
    for m in cf_params:
        if m != '<>':
            val = str(params_info[params_info['code']==m]['P_value'].to_list()[0])
            # Change '' to \'\' in case of string val
            if val.isdigit()==False:
                if '.' in val:
                    v = val.replace('.','')
                    if v.isdigit():
                        val=val
                    else:
                        val=repr(val)
                else:
                    val=repr(val)
            else:
                val=val
            c_formula = c_formula.replace(str(m),val)
        else:
            c_formula = c_formula.replace('<>','!=')

    # Change '=' to '=='
    if '<=' in c_formula:
        c_formula=c_formula.replace('<=','@')
    if '>=' in c_formula:
        c_formula=c_formula.replace('>=',';')
    if '!=' in c_formula:
        c_formula=c_formula.replace('!=','?')
    if '=' in c_formula:
        c_formula = c_formula.replace('=','==')
    if '@' in c_formula:
        c_formula=c_formula.replace('@','<=')
    if ';' in c_formula:
        c_formula=c_formula.replace(';','>=')
    if '?' in c_formula:
        c_formula = c_formula.replace('?','!=')
    return c_formula

In [17]:
#Based on vt_tax, vt_owner, vt_tax_based_on, vt_tax_based_on_permit_info, va_details

date_format = "%d-%b-%Y"
for i in range(0, 21):
    #print(i)
    final_rate=0
    regn_no = vt_tax['Regn No'].to_list()[i]
    state_cd = vt_tax['State Cd'].to_list()[i]
    off_cd = vt_tax['Off Cd'].to_list()[i]
    rcpt_no = vt_tax['Rcpt No'].to_list()[i]
    #print(regn_no,rcpt_no)
    tax_amt = vt_tax['Tax Amt'].to_list()[i]
    rcpt_dt = vt_tax['Rcpt Dt'].to_list()[i]
    
    if relativedelta(datetime.strptime(rcpt_dt[:11], date_format), datetime.strptime(tm_office[(tm_office['Off Cd']==off_cd) & (tm_office['State Cd']==state_cd)]['Vow4'].to_list()[0],'%d-%b-%Y')).days <0 :
        print(i,'Legacy receipt ', rcpt_no,'(',rcpt_dt,') Regn No',regn_no,' collected by',vt_tax['Collected By'].to_list()[i])
        continue
    
    if len(vt_owner[vt_owner['Regn No']==regn_no])>0:
        vh_class = vt_owner[vt_owner['Regn No']==regn_no]['Vh Class'].to_list()[0]
    else:
        print(i,'NO INFO IN VT_OWNER',regn_no)
        continue
    tax_mode = vt_tax['Tax Mode'].to_list()[i]
    tm_flag='y'
    if tax_mode == 'Q':
        tm_flag = 'q'
        tax_mode= 'Y'
    state_cd = vt_tax['State Cd'].to_list()[i]
    pur_cd = vt_tax['Pur Cd'].to_list()[i]
    tax_from = vt_tax['Tax From'].to_list()[i]
    tax_upto = vt_tax['Tax Upto'].to_list()[i]
    
    slab_codes = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) & (slab_new['state_cd']==state_cd) & (slab_new['date_from']<datetime.strptime(rcpt_dt[:11], '%d-%b-%Y')) & (slab_new['date_to']>datetime.strptime(rcpt_dt[:11], '%d-%b-%Y')) ]['slab_code'].to_list()
    #print('Slab Codes Applicable:',slab_codes)
    if len(slab_codes)==0:
        print(i,'Missed',rcpt_no,regn_no,'vh_class',vh_class)
        continue
    formula_df = slab_add_new.loc[(slab_add_new['slab_code'].isin(slab_codes)) & (slab_add_new['state_cd']==state_cd)]
    if (state_cd == 'JH') & (30 in slab_codes) | (84 in slab_codes):
        #print(i,'Type flag \'T\' is changed to \'R\'')
        if (30 in slab_codes):
            formula_df.loc[(formula_df.slab_code==30) & (formula_df.rate_formula.str.contains('<581>')),'type_flag']='R'
        else:
            formula_df.loc[(formula_df.slab_code==84) & (formula_df.rate_formula.str.contains('<581>')),'type_flag']='R'

    
    #Identify the parameters required for calculating the formulas
    params = []
    for j in range(len(formula_df)):
        formula = formula_df['condition_formula'].to_list()[j] +' '+formula_df['rate_formula'].to_list()[j]
        for k in re.split(r'[()\*\s]\s*',formula):
            if '<' in k and '>' in k:
                params.append(k[k.find('<'):(k.find('>')+1)])
    params = list(set(params))
    #print('Required Parameters :',params)
    params_info = slab_fields.loc[slab_fields['code'].isin(params)]
    params_info = params_info.reindex(columns = params_info.columns.tolist() + ['P_value'])
    param_info_tbo = params_info[params_info['table - slab_field']=='vt_tax_based_on']
    param_info_tbopi = params_info[params_info['table - slab_field']=='vt_tax_based_on_permit_info']
    #param_info_misc = params_info[(params_info['table - slab_field']!='vt_tax_based_on_permit_info') & (params_info['table - slab_field']!='vt_tax_based_on')]
    
    # Extract parameters from the TBO table
    if len(param_info_tbo)>0:
        tbo_df = vt_tax_based_on[vt_tax_based_on['regn_no']==regn_no]
        tbo_df= copy.deepcopy(tbo_df)
        if len(tbo_df)==0:
            print(i,'NO INFO IN VT_TBO',rcpt_no,' Regn No:',regn_no)
            continue
        tbo_df['op_dt'] = tbo_df.loc[:,'op_dt'].apply(lambda x: datetime.strptime(x[:11], '%d-%m-%Y'))
        tbo_df = tbo_df.sort_values(by='op_dt', ascending=False)
        #print(tbo_df['op_dt'])
        #Identify the last receipt details in TBO                                                                                                                            
        for n in range(len(tbo_df)):
            x = relativedelta(datetime.strptime(rcpt_dt[:11], '%d-%b-%Y'), datetime.strptime(str(tbo_df['op_dt'].to_list()[n])[:10],'%Y-%m-%d'))
            x = x.years*365 + x.months*30 + x.days
            if x>=0:
                last_rcpt = tbo_df['rcpt_no'].to_list()[n]
                break
            last_rcpt=''
        if last_rcpt=='':
            print(i,'No prior receipt info!!! Rcpt:',rcpt_no,' Regn No:',regn_no)
            continue

        for pi in range(len(param_info_tbo)):
            col = param_info_tbo['column_name'].to_list()[pi][1:-2]
            par = param_info_tbo['column - slab_field'].to_list()[pi]
            code = param_info_tbo['code'].to_list()[pi]
            p_value = ''
            if par=='vch_age':
                p_value = relativedelta(datetime.strptime(rcpt_dt[:11], date_format), datetime.strptime(vt_owner[vt_owner['Regn No']==regn_no]['Purchase Dt'].to_list()[0],date_format))
                p_value = p_value.years
            elif par=='vch_age_months':
                p_value = relativedelta(datetime.strptime(rcpt_dt[:10], date_format), datetime.strptime(vt_owner[vt_owner['Regn No']==regn_no]['Purchase Dt'].to_list()[0],date_format))
                p_value = (p_value.years *12) + p_value.months
            elif par=='purchase_dt_vch_age':
                p_value = relativedelta(datetime.strptime(rcpt_dt[:10], date_format), datetime.strptime(tbo_df[tbo_df['rcpt_no']==last_rcpt]['purchase_dt'].to_list()[0],'%d-%m-%Y'))
                p_value = p_value.years
            elif par=='tax_month_duration':
                p_value = relativedelta(datetime.strptime(tax_upto, date_format), datetime.strptime(tax_from, date_format))
                p_value = p_value.years *12 + p_value.months
            elif par=='distance_run_in_month':
                p_value = vt_tax_based_on['distance_run_in_quarter']/4
            elif par=='transaction_pur_cd':
                appl_no = vp_appl_rcpt_mapping[vp_appl_rcpt_mapping['rcpt_no']==rcpt_no]['appl_no'].to_list()[0]
                if len(va_details[va_details['appl_no']==appl_no][col].to_list())==0:
                    print(i,'No pur_cd available in va_details')
                    print('Appl no',appl_no,' !!! Rcpt:',rcpt_no,' Regn No:',regn_no,'vh_class',vh_class)
                    continue
                else:
                    p_value = va_details[va_details['appl_no']==appl_no][col].to_list()[0]
                #print(p_value)
            elif par=='vch_type':
                p_value = vm_vh_class[vm_vh_class['vh_class']==vh_class]['Class Type'].to_list()[0]
            elif par=='new_vch':
                p_value = vt_owner[vt_owner['Regn No']==regn_no]['Status'].to_list()[0]
            elif par=='tax_mode':
                p_value = tax_mode
            elif par=='delay_days':
                #print(rcpt_dt[:10],' ',tax_from)
                p_value = relativedelta(datetime.strptime(rcpt_dt[:11], '%d-%b-%Y'),datetime.strptime(tax_from, date_format))
                if p_value.days > 0:
                    p_value = p_value.days
                elif p_value.days == 0:
                    p_value = 1
                else:
                    p_value = 0
                #print(p_value)
            else:
                p_value = tbo_df[tbo_df['rcpt_no']==last_rcpt][col].to_list()[0]
            #print(col,' ',par,' ',code,' ',p_value)
            #print(p_value)
            params_info.loc[params_info.index[params_info['code'] == code].tolist()[0],'P_value']=p_value
    #print(params_info)
    #Calculate parameters from TBOPI
    if len(param_info_tbopi)>0:
        tbopi_df = vt_tax_based_on_permit_info[vt_tax_based_on_permit_info['regn_no']==regn_no]
        tbopi_df = tbopi_df.sort_values(by='op_dt', ascending=False)

        for n in range(len(tbopi_df)):
            if rcpt_dt  > tbopi_df['op_dt'][n]:
                last_rcpt = tbopi_df['rcpt_no'][n]
                break
        for pi in range(len(param_info_tbopi)):
            p_value = tbopi_df[tbopi_df['rcpt_no']==last_rcpt][params_info_tbo['column_name']]
            params_info['P_value']=p_value                                                                              
    elif 'service_type' in params_info['column - slab_field'].to_list():
        params_info[params_info['column - slab_field']=='service_type']['P_value']=0

    #Calculate the Formula one by one
    ind=[]
    reb_df = formula_df[formula_df['type_flag']=='R']
    for l in range(len(formula_df)):
        condition_formula = formula_df['condition_formula'].to_list()[l]
        if '\r' in condition_formula:
            condition_formula = condition_formula.replace('\r','')
        if '\n' in condition_formula:
            condition_formula = condition_formula.replace('\n','')

        cf_params = []
        for k in re.split(r'[()\*\s]\s*',condition_formula):
            if '<' in k and '>' in k:
                cf_params.append(k[k.find('<'):(k.find('>')+1)])
        cf_params = list(set(cf_params))
        c_formula = condition_formula
        c_formula = condition_form_convert(cf_params,params_info, c_formula)
        if c_formula.lower() != 'false' and formula_df['type_flag'].to_list()[l] != 'R':
            #print(c_formula)
            cf_result = eval(c_formula.lower())
        else:
            cf_result = False
        if cf_result==True:
            ind.append(l)
    formula_df.reset_index(drop=True, inplace=True)
    
    #Module for calculation of rates for true conditions
    true_df = formula_df.loc[ind,:]
    true_df.reset_index(drop=True, inplace=True)
    t_params = []
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for k in re.split(r'[()\*\s]\s*',r_formula):
            if '<' in k and '>' in k:
                t_params.append(k[k.find('<'):(k.find('>')+1)])

    t_params = list(set(t_params))
    rates=[]
    for q in range(len(true_df)):
        r_formula = true_df['rate_formula'].to_list()[q]
        for m in t_params:
            r_formula = r_formula.replace(str(m),str(params_info[params_info['code']==m]['P_value'].to_list()[0]))
        if 'CEILING' in r_formula: r_formula.replace('CEIL','math.ceil')
        if 'CEIL' in r_formula: r_formula = r_formula.replace('CEIL','math.ceil')
        rate = eval(r_formula.lower())
        rates.append(rate)
        
    true_df['Rate']=rates
    
    #Module for dealing Minimum M and Amount T 
    if len(set(true_df['condition_formula'].to_list()))>1:
        true_df = true_df[true_df['condition_formula']!='1=1']
        if len({'T','M'} - set(true_df['type_flag'].to_list()))==0:
            if true_df[true_df['type_flag']=='T']['Rate'].to_list()[0]> true_df[true_df['type_flag']=='M']['Rate'].to_list()[0]:
                final_rate = true_df[true_df['type_flag']=='T']['Rate'].to_list()[0]
            else:
                final_rate = true_df[true_df['type_flag']=='M']['Rate'].to_list()[0]
    else:
        final_rate = true_df['Rate'].to_list()[0] 
        
    #Module for Rebate
    if len(reb_df)>0 and tm_flag !='q':
        reb_params = []
        for q in range(len(reb_df)):
            reb_formula = reb_df['condition_formula'].to_list()[q] +' '+reb_df['rate_formula'].to_list()[q]
            for k in re.split(r'[()\*\s]\s*',reb_formula):
                if '<' in k and '>' in k:
                    reb_params.append(k[k.find('<'):(k.find('>')+1)])
        #print(reb_params)
        for e in reb_params:
            if e=='<581>':
                p_value = final_rate
                params_info.loc[params_info['code'] == e,'P_value']=p_value
        for f in reb_df['condition_formula'].to_list():
            r_formula = f
            rr_formula = reb_df[reb_df['condition_formula']==f]['rate_formula'].to_list()[0]
            r_formula = condition_form_convert(reb_params,params_info,r_formula)
            reb_result = eval(r_formula.lower())
            if reb_result == True:
                ra_formula = condition_form_convert(reb_params,params_info,rr_formula)
                reb_rate_result = eval(ra_formula.lower())              
                final_rate = final_rate - reb_rate_result
            
    #Modeule for Tax Mode Remaining years
    '''
    if tax_mode in ['Y15','Y10','Y12','Y20']:
        tax_period = relativedelta(datetime.strptime(tax_upto, date_format), datetime.strptime(tax_from, date_format))
        tax_period = tax_period.years*365 + tax_period.months*30 + tax_period.days
        veh_age = relativedelta(datetime.strptime(tax_from, date_format), datetime.strptime(vt_owner[vt_owner['Regn No']==regn_no]['Purchase Dt'].to_list()[0],'%Y-%m-%d'))
        tax_from_slab_codes = slab_new[(slab_new['vh_class']==vh_class) & (slab_new['pur_cd']==pur_cd) & (slab_new['tax_mode']==tax_mode) & (slab_new['state_cd']==state_cd) & (slab_new['date_from']<datetime.strptime(tax_from, date_format)) & (slab_new['date_to']>datetime.strptime(tax_from, date_format)) ]['slab_code'].to_list()
        if len(set(slab_codes)-set(tax_from_slab_codes))!=0:
            
        
        else:
            if tax_mode=='Y10':
                days_rem = 3652 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
                final_rate = (final_rate/3652)*days_rem
            if tax_mode=='Y12':
                days_rem = 4383 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
                #print('Days remaining in ',tax_mode,'=',days_rem,'days')
                final_rate = (final_rate/4383)*days_rem
            if tax_mode=='Y15':
                days_rem = 5478 - ((veh_age.years * 365) + (veh_age.months * 30) + veh_age.days)
                final_rate = (final_rate/5478)*days_rem
    '''                            
    #Module for Quarterly Mode    
    if tm_flag=='q':
        tax_period = relativedelta(datetime.strptime(tax_upto, date_format), datetime.strptime(tax_from, date_format))
        years_q = math.ceil(tax_period.years * 4)
        #tax_period = tax_period.years*12 + tax_period.months
        if tax_period.months <=3:
            months_q = 1
        elif tax_period.months <=6:
            months_q = 2
        elif tax_period.months <=9:
            months_q = 3
        else:
            months_q = 4
        n_quart = years_q + months_q
        final_rate = (final_rate/4)*n_quart

    #Display Calculated Amount
    if int(tax_amt) - int(math.ceil(final_rate)) > 1 or int(tax_amt) - int(math.ceil(final_rate)) < -1:
        print(i,'MISMATCH!!! Rcpt:',rcpt_no,' Regn No:',regn_no,'vh_class',vh_class)
        print('Tax Amt: Rs',tax_amt,' Calculated: Rs',int(math.ceil(final_rate)))
    else:
        print(rcpt_no,'Tax Amt: Rs',tax_amt,' Calculated: Rs',int(math.ceil(final_rate)))

ValueError: time data '23-Dec-201' does not match format '%Y-%m-%d'

In [23]:
vt_tax_based_on['op_dt'].head(5)

0    08-02-2017 08:52:18
1    01-01-2019 17:58:21
2    20-07-2019 19:18:36
3    29-03-2017 13:16:45
4    09-11-2017 17:57:17
Name: op_dt, dtype: object

In [19]:
formula_df

,add_code,condition_formula,emp_cd,op_dt,rate_formula,slab_code,state_cd,type_flag
0,1,<2><=500,200020,17-03-2015 10:21,298.5,4,JH,T
1,2,<2>>500 AND <2><=2000,200020,02-07-2015 14:46,298.50+(CEIL((<2>-500)/250.0)*34),4,JH,T
2,3,<2>>2000 AND <2><=4000,200020,02-07-2015 14:45,502.50+(CEIL((<2>-2000)/250.0)*42),4,JH,T
3,4,<2>>4000 AND <2><=8000,200020,02-07-2015 14:45,838.50+(CEIL((<2>-4000)/250.0)*51.50),4,JH,T
4,5,<2>>8000,200020,02-07-2015 14:41,1662.50+(CEIL((<2>-8000)/250.00)*136.50),4,JH,T
5,6,<59>='Y' AND <57> =0,200020,16-09-2015 15:07,<581> *0.05,4,JH,R


In [24]:
true_df

,add_code,condition_formula,emp_cd,op_dt,rate_formula,slab_code,state_cd,type_flag,Rate
0,4,<2>>4000 AND <2><=8000,200020,02-07-2015 14:45,838.50+(CEIL((<2>-4000)/250.0)*51.50),4,JH,T,941.5


In [25]:
final_rate

894.425

In [26]:
reb_df

,add_code,condition_formula,emp_cd,op_dt,rate_formula,slab_code,state_cd,type_flag
9946,6,<59>='Y' AND <57> =0,200020,16-09-2015 15:07,<581> *0.05,4,JH,R


In [22]:
slab_codes

[4]

In [44]:
vt_tax[vt_tax['Regn No']=='JH05BQ1717']

,Regn No,Tax Mode,Payment Mode,Tax Amt,Tax Fine,Rcpt No,Rcpt Dt,Tax From,Tax Upto,Pur Cd,Flag,Collected By,State Cd,Off Cd
42,JH05BQ1717,Y15,I,7647,675,JH190928C2114771,2019-09-28 19:26:37.629217,2019-02-21,2031-11-18,58,NaN,ONLINE TAX,JH,5


In [ ]:
vt_tax[vt_tax['Regn No']=='JH05AF9610']

In [31]:
vt_owner[vt_owner['Regn No']=='JH02U7130']['Vh Class']

2895238    57
Name: Vh Class, dtype: int64

In [465]:
vp_appl_rcpt_mapping[vp_appl_rcpt_mapping['rcpt_no']==rcpt_no]
#p_value = va_details[va_details['appl_no']==appl_no][col].to_list()[0]

,state_cd,off_cd,appl_no,rcpt_no,owner_name,chasi_no,vh_class,instrument_cd,excess_amt,cash_amt,remarks
3843801,JH,5.0,JH190928V1951379,JH190928C2114771,MD.SALAUDDIN,MCG000GP4J1649965,57.0,NaN,0.0,8322.0,NaN


In [45]:
vt_owner[vt_owner['Regn No']==regn_no]['Purchase Dt']

4370206    2011-01-14
Name: Purchase Dt, dtype: object

In [23]:
#Output Dataframe
cff_formula = []
cc_formula=[]
ccf_result=[]
cr_rate=[]
c_add_code=[]
for l in range(len(formula_df)):
    condition_formula = formula_df['condition_formula'].to_list()[l]
    if '\r' in condition_formula:
        condition_formula = condition_formula.replace('\r','')
    if '\n' in condition_formula:
        condition_formula = condition_formula.replace('\n','')

    cf_params = []
    for k in re.split(r'[()\*\s]\s*',condition_formula):
        if '<' in k and '>' in k:
            cf_params.append(k[k.find('<'):(k.find('>')+1)])
    cf_params = list(set(cf_params))
    c_formula = condition_formula
    c_formula = condition_form_convert(cf_params,params_info, c_formula)
    if c_formula.lower() != 'false' and formula_df['type_flag'].to_list()[l] != 'R':
        #print(c_formula)
        cf_result = eval(c_formula.lower())
    else:
        cf_result = False
    if cf_result==True:
        ind.append(l)
    #print(c_formula,cf_result,formula_df.loc[:,'rate_formula'].to_list()[l])
    #if cf_result==True:
    #    ind = l
    cff_formula.append(condition_formula)
    cc_formula.append(c_formula)
    ccf_result.append(cf_result)
    cr_rate.append(formula_df.loc[:,'rate_formula'].to_list()[l])
    #c_add_code.append(add_code)
    #print(c_formula,cf_result,formula_df.loc[:,'rate_formula'].to_list()[l])
#print('Identified Parameters: ',params_info)
df = pd.DataFrame(list(zip(cff_formula,cc_formula, ccf_result,cr_rate)), columns =['Condition Formula', 'Condition Result','Result','Rate Formula']) 
df

,Condition Formula,Condition Result,Result,Rate Formula
0,<2><=500,4450.0<=500,False,298.5
1,<2>>500 AND <2><=2000,4450.0>500 AND 4450.0<=2000,False,298.50+(CEIL((<2>-500)/250.0)*34)
2,<2>>2000 AND <2><=4000,4450.0>2000 AND 4450.0<=4000,False,502.50+(CEIL((<2>-2000)/250.0)*42)
3,<2>>4000 AND <2><=8000,4450.0>4000 AND 4450.0<=8000,True,838.50+(CEIL((<2>-4000)/250.0)*51.50)
4,<2>>8000,4450.0>8000,False,1662.50+(CEIL((<2>-8000)/250.00)*136.50)
5,<59>='Y' AND <57> =0,'Y'=='Y' AND 0 ==0,False,<581> *0.05


In [516]:
true_df

,add_code,condition_formula,emp_cd,op_dt,rate_formula,slab_code,state_cd,type_flag,Rate
0,1,<5> >0,200120,03-01-2019 11:44,<5>*0.07,43,JH,T,13440.0
